# Report for Banana Collector solved by a DQN algorithm
## Code implementation description
The whole code contains four python files:
1. `model.py` contains a class __Actor__, which is a neural network (NN) with two hidden layers (37,64,64,4), so this NN codifies the policy.
1. `dqn_agent.py` has a class __Agent__ with an __Actor__ class and __ReplayBuffer__ class as members, Agent has three important methods `act`, `step` and `learn`. Furthermore __ReplayBuffer__ is an implementation of the Expirience Replay technique propose in the DQN algorithm.
1. `dqn_monitor.py` has a function `dqn_interact` which implements the interacion of the Unity environment and the class __Agent__ following the DQN algorithm.
1. `learn_and_prove.py` is the main program and has four options `[-h|--help]` to know how to use the program, `[--train]` to train an agent, `[--random]` to run a tabula rasa or a random-policy agent, and `[--file FILE]` to save the output data from the program for a post-process. 

## Learning algorithm

### DQN algorithm explanation
The __DQN algorithm__ takes the following function to let the Agent learn from experience, here is performed the $\epsilon_i$ management under GLIE technique for the _exploitation-exploration dilemma_ and a $\epsilon$-greedy policy function is accomplished under `agent.act(...)`,

```python
def dqn_interact(env, agent,
                 n_episodes=2000, window=100, max_t=1000,
                 eps_start=1.0, eps_end=0.005, eps_decay=0.980,
                 filename='checkpoint.pth'):
    """ Deep Q-Learning Agent-Environment interaction.
    
    Params
    ======
        env: instance of UnityEnvironment class
        agent: instance of class Agent (see dqn_agent.py for details)
        n_episodes (int): maximum number of training episodes
        window (int): number of episodes to consider when calculating average rewards
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        filename (string): name of the file to save weights
    """
    # all returns
    all_returns = []
    # initialize average rewards
    avg_rewards = deque(maxlen=n_episodes)
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # initialize best average reward
    best_avg_reward = -np.inf
    # initialize eps
    eps = eps_start
    # for each episode
    for i_episode in range(1, n_episodes+1):
        # begin the episode
        state = reset(env, train_mode=True)
        # initialize the sample reward
        samp_reward = 0
        for t in range(max_t):
            # agent selects an action
            action =  agent.act(state, eps)
            # agent performs the selected action
            next_state, reward, done = step(env, action)
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)
            # updated the sample reward
            samp_reward += reward
            # update the state (s-> s') to next time step
            state = next_state
            if done:
                break
        # save final sampled reward
        samp_rewards.append(samp_reward)
        all_returns.append(samp_reward)
        # update epsion with GLIE
        eps = max(eps_end, eps_decay*eps)
        # stopping criteria
        if np.mean(samp_rewards)>=15.0:
            # safe weights
            break

    return

```
Inside `agent.step(...)` method is carried out the learning process, the agent begin in a [_tabula rasa_](https://en.wikipedia.org/wiki/Tabula_rasa) state, after that each interaction with the environment is saved in the replay buffer and only when the replay buffer is fulfilled with at least `BATCH_SIZE` of experiences the learning process begin with a sample of randomly `BUFFER_SIZE` experiences, this sample is performed in the ReplayBuffer.sample() method.

```python
class Agent:
 
    # ...
    def step(self, state, action, reward, next_state, done):
        """ Interact Agent and Environment.
        """
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step ==0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

```

The `learn(...)` method called from `agent.step(...)` performs the TD-update using a target NN as proposed in [this paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) but with a little change in the update step of the target NN using a `soft_update` method proposed in the [paper](https://arxiv.org/pdf/1509.02971.pdf),

```python
class Agent:
    
    # ...
    def learn(self, experiences, gamma):
        """ Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.actor_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.actor_local(states).gather(1,actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        self.soft_update(self.actor_local, self.actor_target, TAU)    
```

### Hyperparameters exploration
The following important parameters are handled by the following dictionary:

```python
# current configuration
config = {
    'n_episodes':       400,  # max. number of episode to train the agent
    'window':           100,  # save the last XXX returns of the agent
    'max_t':            500,  # max. number of steps per episode
    'eps_start':        1.0,  # GLIE parameters
    'eps_end':        0.005,  #  - epsilon minimum value
    'eps_decay':      0.960,  #  - epsilon decay 
    'BUFFER_SIZE': int(1e5),  # replay buffer size
    'BATCH_SIZE':        16,  # minibatch size
    'GAMMA':           0.99,  # discount factor
    'TAU':             1e-3,  # for soft update or target parameters
    'LR':              5e-4,  # learning rate
    'UPDATE_EVERY':       1,  # how often to update the network
    'FC1_UNITS':         16,  # number of neurons in fisrt layer
    'FC2_UNITS':         16,  # number of neurons in second layer
    }
```

* `n_episodes` was explored from a range $\in [100-2000]$ and was concluded that 400 episodes was enough.
* `max_t` was explored from a range $\in [100-1000]$ and 500 was chosen because was the minimum amount to let the agent explores close the wall and get that experience without having to wait long episodes.
* `eps_decay` was expored with values like $[0.999, 0.99, 0.98, 0.96]$. At the end we pick $\epsilon_d=0.96$.
* `BATCH_SIZE` seems to be not so important. We tried with this values $[16,32,64]$ but nothing important happend, then we chose simplicity.
* `GAMMA` was explored with 0.99 and 0.999. It seems better to chose 0.99.
* `UPDATE_EVERY` makes learning time to be prolonged.
* `FC1_UNITS` and `FC2_UNITS` was explored with values like $[16,32,64]$, finally was chosen 16 for simplicity.

The other parameters was not explored. Finally, we conclude the most important parameter found was `eps_decay`.

### The neural networks architecture
The final architecture for simplicity was 37->16->16->4,

```python
Actor(
  (fc1): Linear(in_features=37, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=4, bias=True)
)

```

the definition can be found in `model.py`, the activation function for hidden layers are `relu` functions,

```python
class Actor(nn.Module):
    """ Actor (Policy) Model. """

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """ Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """ Build a network that maps state -> action values. """
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
```

## Plots of rewards
Reports the number of episodes needed to solve the environment

When the code run the following graph can be build,
![title](images/drl01_simple_run.png)
all agent's learning seasons have a target of +13 avg score over the last 100 episodes (red line), an upper goal of +15 avg. score over the last 100 episodes (green line) is set in the algorithm to finish the main loop. The grey curve is all the final scores obtained by the 2000 episodes of agent experience. The blue line represents the avg. score over the last 100 scores.

The following images show the effect of $\epsilon_d$ and the reason why could be a good choice of $\epsilon_d=0.960$; furthermore, `max_t=500` and `n_episodes` will be set for the rest of the analysis. 
![title](images/drl02_exploring_epsilon_decay.png)
![title](images/drl03_how_epsilon_decay.png)

The number neurons of the hidden layers was modified and finally simplicity was selected.
![title](images/drl04_exploring_fci.png)

The stochasticity of the problem makes hard to explore all the paramiters, the following figure has created holding the parameters for five runs and then was obtained the average and standard deviation of episodes to solve the problem.
![title](images/drl05_stochasticity_and_gamma.png)
```python
Episodes solved avg. (avg:323.60 | std:46.81) for epsilon_d = 0.980 and gamma = 0.99
Episodes solved avg. (avg:217.20 | std:17.44) for epsilon_d = 0.960 and gamma = 0.99
Episodes solved avg. (avg:225.60 | std:14.01) for epsilon_d = 0.960 and gamma = 0.999
Episodes solved avg. (avg:234.80 | std:24.77) for epsilon_d = 0.960 and gamma = 0.96
```
Also the $\gamma$ effect was explored and its relation with the standard deviation of the learning curve. Nonetheless we thing we need to do more proves.

## Ideas for future work

For future work its important to make a better systematic search on the parameter space. In the case of DQN algorithm, this can be improved with other improvements suggested in [the paper](https://arxiv.org/abs/1710.02298) with modifications like Double DQN, Prioritized Experience Replay, Dueling DQN and others.